<a href="https://colab.research.google.com/github/A1DS19/Movie-review-classifier-BIRNN/blob/master/IMDB_Classification_Simplest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow import keras

!nvidia-smi -L

GPU 0: Tesla P100-PCIE-16GB (UUID: GPU-2b52b37e-bd0e-4e81-7b06-fd71f60ce004)


### Load data

In [3]:
imdb_train, ds_info = tfds.load('imdb_reviews',
                                split='train',
                                with_info=True,
                                as_supervised=False)
ds_info

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Shuffling ~/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete86FQPC/imdb_reviews-train.tfrecord*...…

...

Shuffling ~/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete86FQPC/imdb_reviews-test.tfrecord*...:…

Generating unsupervised examples...:   0%|          | 0/50000 [00:00<?, ? examples/s]

Shuffling ~/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete86FQPC/imdb_reviews-unsupervised.tfrec…

Dataset imdb_reviews downloaded and prepared to ~/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


tfds.core.DatasetInfo(
    name='imdb_reviews',
    full_name='imdb_reviews/plain_text/1.0.0',
    description="""
    Large Movie Review Dataset.
    This is a dataset for binary sentiment classification containing substantially more data than previous benchmark datasets. We provide a set of 25,000 highly polar movie reviews for training, and 25,000 for testing. There is additional unlabeled data for use as well.
    """,
    config_description="""
    Plain text
    """,
    homepage='http://ai.stanford.edu/~amaas/data/sentiment/',
    data_path='~/tensorflow_datasets/imdb_reviews/plain_text/1.0.0',
    file_format=tfrecord,
    download_size=80.23 MiB,
    dataset_size=129.83 MiB,
    features=FeaturesDict({
        'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=2),
        'text': Text(shape=(), dtype=tf.string),
    }),
    supervised_keys=('text', 'label'),
    disable_shuffling=False,
    splits={
        'test': <SplitInfo num_examples=25000, num_shards=1>,
        '

In [4]:
imdb_test = tfds.load('imdb_reviews',
                      split='test',
                      as_supervised=True)

In [ ]:
BATCH_SIZE = 32
imdb_train = imdb_train.batch(BATCH_SIZE)
imdb_test = imdb_test.batch(BATCH_SIZE)

In [13]:
# negative review
for example, label in imdb_train.take(1):
    print(example, '\n', label)

tf.Tensor(
[b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it."
 b'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell asleep because the film was rubbish. The plot de

In [23]:
imdb_txt_train = imdb_train.map(lambda text, label:text)
imdb_label_train = imdb_train.map(lambda text, label:label)

imdb_txt_test = imdb_test.map(lambda text, label:text)
imdb_label_test = imdb_test.map(lambda text, label:label)

In [24]:
for example in imdb_txt_train.take(1):
    print(example[:10])

tf.Tensor(
[b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it."
 b'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell asleep because the film was rubbish. The plot de

In [25]:
for example in imdb_label_train.take(1):
    print(example[:10])

tf.Tensor([0 0 0 1 1 1 0 0 0 0], shape=(10,), dtype=int64)


In [26]:
text_vectorization = keras.layers.TextVectorization(output_mode='int',
                                                    split='whitespace',
                                                    max_tokens=20000)

In [28]:
text_vectorization.adapt(imdb_txt_train)

In [29]:
test_txt = [
    "Hello my name is",
    "I'm happy",
    "What time is it"
]

text_vectorization(test_txt)

<tf.Tensor: shape=(3, 4), dtype=int64, numpy=
array([[5010,   56,  398,    7],
       [ 142,  673,    0,    0],
       [  49,   62,    7,    9]])>

In [36]:
from tensorflow.keras import layers

def create_model():
    inputs = layers.Input(shape=(1,), dtype=tf.string)
    x = text_vectorization(inputs)
    x = layers.Embedding(input_dim=20000, output_dim=256)(x)
    x = layers.Bidirectional(layers.LSTM(64))(x)
    outputs = layers.Dense(1, activation='sigmoid')(x)
    model = keras.Model(inputs, outputs)
    model.compile(loss=keras.losses.binary_crossentropy,
                  optimizer=keras.optimizers.Adam(),
                  metrics=['accuracy', 'Precision', 'Recall'])
    return model

In [37]:
model = create_model()

In [39]:
model.fit(imdb_train,
          epochs=10)

Epoch 1/10
782/782 [==============================] - 58s 65ms/step - loss: 0.4768 - accuracy: 0.7817 - precision: 0.7692 - recall: 0.8049
Epoch 2/10
782/782 [==============================] - 51s 65ms/step - loss: 0.3366 - accuracy: 0.8624 - precision: 0.8633 - recall: 0.8613
Epoch 3/10
782/782 [==============================] - 51s 65ms/step - loss: 0.2683 - accuracy: 0.8956 - precision: 0.9105 - recall: 0.8774
Epoch 4/10
782/782 [==============================] - 51s 65ms/step - loss: 0.1808 - accuracy: 0.9354 - precision: 0.9389 - recall: 0.9314
Epoch 5/10
782/782 [==============================] - 51s 65ms/step - loss: 0.1185 - accuracy: 0.9599 - precision: 0.9635 - recall: 0.9560
Epoch 6/10
782/782 [==============================] - 51s 66ms/step - loss: 0.0912 - accuracy: 0.9716 - precision: 0.9733 - recall: 0.9698
Epoch 7/10
782/782 [==============================] - 51s 66ms/step - loss: 0.0679 - accuracy: 0.9788 - precision: 0.9798 - recall: 0.9777
Epoch 8/10
782/782 [=======

In [40]:
model.evaluate(imdb_test)

782/782 [==============================] - 23s 28ms/step - loss: 0.7383 - accuracy: 0.8165 - precision: 0.8549 - recall: 0.7625


[0.7382620573043823, 0.8165199756622314, 0.854874849319458, 0.7624800205230713]

In [71]:
review = """
I watch a lot of movies and I like to give them all a chance just in case there is something interesting or exciting to warrant a viewing Unfortunately this movie has none of these features it is pointless and offers nothing in the way of story line,acting or direction The plot is non-existent with the actors just going through the motions and the dialogue is sooo boring its embarrassing. I wish the previous reviewers had posted earlier as this would have saved me 95 mins of my time
"""

pred = model.predict([review])
res = pred[0][0]
print('negative' if res < 0.5 else 'positive')
print('pred', res)

negative
pred 0.00041814742
